In [3]:
from keras import optimizers
from keras.datasets import cifar10
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import warnings
warnings.filterwarnings('ignore')

In [4]:
width, height = 32, 32
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

epoch = 40
classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, classes)
y_test = np_utils.to_categorical(y_test, classes)

train_samples = x_train.shape[0]
validation_samples = x_test.shape[0]

In [6]:
# Extract the last layer from third block of vgg16 model
#https://github.com/keras-team/keras/issues/4040
last = base_model.get_layer('block3_pool').output
# Add new layers on top of it
#https://stackoverflow.com/questions/48708870/kerasunable-to-add-dense-layer-to-vgg16
top = Flatten()(last)
top = Dense(256, activation='relu')(top)
top = Dropout(0.5)(top)
pred = Dense(10, activation='sigmoid')(top)

model = Model(base_model.input, pred)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [7]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow(x_test, y_test, batch_size=32)


In [8]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=train_samples,
    nb_epoch=epoch,
    validation_data=validation_generator,
    nb_val_samples=validation_samples,
    )


Epoch 1/40
1562/1562 [==============================] - 54s 35ms/step - loss: 1.6183 - acc: 0.8992 - val_loss: 1.6118 - val_acc: 0.9000
Epoch 2/40
1562/1562 [==============================] - 51s 33ms/step - loss: 1.6118 - acc: 0.9000 - val_loss: 1.6118 - val_acc: 0.9000
Epoch 3/40
1562/1562 [==============================] - 51s 33ms/step - loss: 1.6064 - acc: 0.8994 - val_loss: 0.5080 - val_acc: 0.8947
Epoch 4/40
1562/1562 [==============================] - 51s 33ms/step - loss: 0.2610 - acc: 0.9055 - val_loss: 0.1956 - val_acc: 0.9247
Epoch 5/40
1562/1562 [==============================] - 52s 33ms/step - loss: 0.2055 - acc: 0.9214 - val_loss: 0.1638 - val_acc: 0.9365
Epoch 6/40
1562/1562 [==============================] - 52s 33ms/step - loss: 0.1812 - acc: 0.9304 - val_loss: 0.1466 - val_acc: 0.9420
Epoch 7/40
1562/1562 [==============================] - 53s 34ms/step - loss: 0.1662 - acc: 0.9365 - val_loss: 0.1356 - val_acc: 0.9473
Epoch 8/40
1562/1562 [==========================

In [9]:
# save the model
model.save('weights.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.7049338786602021
Test accuracy: 0.9541199987411499
